In [57]:
import pandas as pd

# Load datasets
recipes_df = pd.read_csv('C:/Users/arsen/Healthylicious/data/cleaned/csv/recipes_dataset.csv')
ratings_df = pd.read_csv('C:/Users/arsen/Healthylicious/data/cleaned/csv/ratings_dataset.csv')


In [58]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

user_encoder = LabelEncoder()
recipe_encoder = LabelEncoder()

ratings_df['user'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['recipe'] = recipe_encoder.fit_transform(ratings_df['recipeId'])

num_users = ratings_df['user'].nunique()
num_recipes = ratings_df['recipe'].nunique()

train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42)

In [59]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.regularizers import l2

# Input layers
user_input = Input(shape=(1,), name='user_input')
recipe_input = Input(shape=(1,), name='recipe_input')

# Embedding layers
user_embedding = Embedding(input_dim=num_users, output_dim=50, name='user_embedding',
                           embeddings_regularizer=l2(1e-6))(user_input)
recipe_embedding = Embedding(input_dim=num_recipes, output_dim=50, name='recipe_embedding',
                             embeddings_regularizer=l2(1e-6))(recipe_input)

# Flatten the embedding layers
user_vec = Flatten(name='user_flatten')(user_embedding)
recipe_vec = Flatten(name='recipe_flatten')(recipe_embedding)

# Concatenate user and recipe vectors
concat = Concatenate()([user_vec, recipe_vec])

# Dense layers with dropout
dense = Dense(128, activation='relu')(concat)
dense = Dropout(0.5)(dense)
dense = Dense(64, activation='relu')(dense)
dense = Dropout(0.5)(dense)
output = Dense(1)(dense)

# Create the model
model = Model(inputs=[user_input, recipe_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Prepare the data for training
X_train = [train_df['user'], train_df['recipe']]
y_train = train_df['rating']

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)


Epoch 1/50
1001/1001 [==============================] - 3s 2ms/step - loss: 1.8597 - val_loss: 0.9717
Epoch 2/50
1001/1001 [==============================] - 2s 2ms/step - loss: 1.2917 - val_loss: 0.9596
Epoch 3/50
1001/1001 [==============================] - 2s 2ms/step - loss: 1.1925 - val_loss: 0.9438
Epoch 4/50
1001/1001 [==============================] - 2s 2ms/step - loss: 1.1143 - val_loss: 0.9401
Epoch 5/50
1001/1001 [==============================] - 2s 2ms/step - loss: 1.0505 - val_loss: 0.9408
Epoch 6/50
1001/1001 [==============================] - 2s 2ms/step - loss: 1.0024 - val_loss: 0.9447
Epoch 7/50
1001/1001 [==============================] - 2s 2ms/step - loss: 0.9616 - val_loss: 0.9481
Epoch 8/50
1001/1001 [==============================] - 2s 2ms/step - loss: 0.9336 - val_loss: 0.9411
Epoch 9/50
1001/1001 [==============================] - 2s 2ms/step - loss: 0.9050 - val_loss: 0.9472
Epoch 10/50
1001/1001 [==============================] - 2s 2ms/step - loss: 0.879

In [60]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare the data for testing
X_test = [test_df['user'], test_df['recipe']]
y_test = test_df['rating']

# Predict ratings for the test set
predictions = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE: {rmse}')


626/626 [==============================] - 1s 766us/step
RMSE: 1.0323378845107818


In [61]:
model.save('collaborative_model.h5')

c:\Users\arsen\Healthylicious\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### START HERE

In [62]:
from tensorflow.keras.models import load_model

model = load_model('collaborative_model.h5')

In [66]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
recipes_df = pd.read_csv('C:/Users/arsen/Healthylicious/data/cleaned/csv/recipes_dataset.csv')
recipes_df_0 = pd.read_csv('C:/Users/arsen/Healthylicious/data/cleaned/csv/recipes_cleaned_with_ids.csv')


# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(recipes_df['Ingredients'])

def get_content_based_recommendations(user_ingredients, top_n=10):
    user_ingredients = [" ".join(user_ingredients)]
    user_tfidf = tfidf_vectorizer.transform(user_ingredients)
    cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix)
    similar_indices = cosine_sim.argsort().flatten()[::-1][:top_n]
    return recipes_df.iloc[similar_indices]

def get_tensorflow_recommendations(user_id, top_n=10):
    user_idx = user_encoder.transform([user_id])[0]
    recipe_ids = np.arange(num_recipes)
    
    # Predict ratings for all recipes for the given user
    user_array = np.array([user_idx] * num_recipes)
    predictions = model.predict([user_array, recipe_ids])
    
    # Get top N recommendations
    top_indices = predictions.flatten().argsort()[-top_n:][::-1]
    recommended_recipe_ids = recipe_encoder.inverse_transform(top_indices)
    
    return recipes_df[recipes_df['recipeId'].isin(recommended_recipe_ids)]

def get_hybrid_recommendations(user_ingredients, user_id, top_n=10):
    # Content-based recommendations
    content_recs = get_content_based_recommendations(user_ingredients, top_n)
    
    # Collaborative recommendations
    collaborative_recs = get_tensorflow_recommendations(user_id, top_n * 2)  # Ambil lebih banyak untuk seleksi
    
    # Filter collaborative recommendations by user ingredients
    user_ingredients_set = set(user_ingredients)
    collaborative_recs_filtered = collaborative_recs[
        collaborative_recs['Ingredients'].apply(lambda x: bool(user_ingredients_set & set(x.split(', '))))
    ]
    
    # Combine and deduplicate recommendations
    combined_recs = pd.concat([content_recs, collaborative_recs_filtered]).drop_duplicates('recipeId')
    
    return combined_recs.head(top_n)

# Example usage
user_ingredients = ['corn']
user_id = 1  # Example user ID
hybrid_recommendations = get_hybrid_recommendations(user_ingredients, user_id)
detailed_data = recipes_df_0[recipes_df_0['recipeId'].isin(hybrid_recommendations['recipeId'])]
print(detailed_data)


35/35 [==============================] - 0s 763us/step
      recipeId   Category                                    Title  \
116        117      Snack    Homemade Cracker Jacks Popcorn Recipe   
272        273  Side Dish                           Corn Casserole   
427        428    Dessert          Homemade Vanilla Pudding Recipe   
473        474    Dessert                      Pastry Cream Recipe   
524        525    Dessert              Poured Fondant Icing Recipe   
553        554    Dessert                     Caramel Glaze Recipe   
588        589    Dessert                          Divinity Recipe   
590        591      Sauce         Praline Sauce Recipe + Printable   
610        611      Sauce  Fresh Cherry Sauce Recipe | Add a Pinch   
1018      1019  Side Dish                Sweet Potato Fries Recipe   

      Total Time                                    All Ingredients  \
116           70  2 cups salted red skinned peanuts, 15 cups pop...   
272           55  6 tablespoons 

### DEBUGGING and VERIFYING

In [55]:
def get_hybrid_recommendations(user_ingredients, user_id, top_n=10):
    # Content-based recommendations
    content_recs = get_content_based_recommendations(user_ingredients, top_n)
    print("Content-Based Recommendations:\n", content_recs.head())
    
    # Collaborative recommendations
    collaborative_recs = get_tensorflow_recommendations(user_id, top_n * 2)  # Ambil lebih banyak untuk seleksi
    print("Collaborative Recommendations:\n", collaborative_recs.head())
    
    # Filter collaborative recommendations by user ingredients
    user_ingredients_set = set(user_ingredients)
    collaborative_recs_filtered = collaborative_recs[
        collaborative_recs['Ingredients'].apply(lambda x: bool(user_ingredients_set & set(x.split(', '))))
    ]
    
    # Combine and deduplicate recommendations
    combined_recs = pd.concat([content_recs, collaborative_recs_filtered]).drop_duplicates('recipeId')
    print("Combined Recommendations:\n", combined_recs.head())
    
    return combined_recs.head(top_n)

# Example usage
user_ingredients = ['cranberry']
user_id = 1  # Example user ID
hybrid_recommendations = get_hybrid_recommendations(user_ingredients, user_id)
print(hybrid_recommendations)


Content-Based Recommendations:
      recipeId     Category                               Title  Total Time  \
49         50      Dessert          Sugared Cranberries Recipe          63   
936       937  Main Course  Cranberry Orange Glazed Ham Recipe          95   
246       247    Breakfast     Cranberry Orange Muffins Recipe          30   
619       620      Dessert   Cranberry Pineapple Sorbet Recipe          15   
71         72    Appetizer       Orange Cranberry Salsa Recipe          10   

                                          Instructions  \
49   Combine 1/2 cup sugar and 1/2 cup water in a m...   
936  Preheat oven to 325º F.\r\nPlace ham into roas...   
246  Preheat oven to 375º F. Spray muffin tins with...   
619  Combine ingredients and make according to your...   
71   Pulse cranberries and sugar together in a food...   

                                             Nutrition   Cuisine       Yields  \
49   {'calories': '164 kcal', 'carbohydrateContent'...  American   8 